In [1]:
!pip install transformers datasets arabert


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 9.5 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=113b9bfbac71b93a3ab98cf533c2ca5c00ebd9bff895f6c30c7550c0e3ab34af
  Stored in directory: /root/.cache/pip/wheels/94/08/b4/78657b1541bb704b088317b52429ee4016d9888fe47dbb130f
Successfully built emoji


In [2]:
!pip install transformers datasets arabert nltk -q


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch
import pandas as pd

In [17]:
from google.colab import files

uploaded = files.upload()



Saving arabic_mh_emotion.csv to arabic_mh_emotion.csv


In [18]:
import pandas as pd

df = pd.read_csv('arabic_mh_emotion.csv')

# Map string labels to numbers
label_map = {'fear': 0, 'sadness': 1, 'anger': 2, 'joy': 3, 'neutral': 4}
df['label'] = df['label'].map(label_map)

texts = df['text'].tolist()
labels = df['label'].tolist()

print(df.head())


                                           text  label
0                       أشعر بالخوف من المستقبل      0
1      لم أستطع النوم الليلة الماضية بسبب القلق      0
2                  قلبي ينبض بسرعة وأنا قلق جدا      0
3                   أشعر بالخوف من فقدان أحبائي      0
4  لا أستطيع التوقف عن التفكير في الأمور السيئة      0


In [19]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)


In [20]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, id2label=id2label, label2id=label2id)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
dataset = dataset.train_test_split(test_size=0.2)

In [23]:
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

<ipython-input-24-2a56602ca032>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
trainer.train()

Step,Training Loss
10,1.672900
20,1.478700
30,1.397400
40,1.267200
50,1.238400


TrainOutput(global_step=50, training_loss=1.4109054756164552, metrics={'train_runtime': 26.6145, 'train_samples_per_second': 3.757, 'train_steps_per_second': 1.879, 'total_flos': 770853933000.0, 'train_loss': 1.4109054756164552, 'epoch': 5.0})

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU if available

for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to same device
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits).item()
    print(f"Input: {text}")
    print(f"Predicted Emotion: {id2label[pred]}\n")


Input: أشعر بالخوف من المستقبل
Predicted Emotion: sadness

Input: لم أستطع النوم الليلة الماضية بسبب القلق
Predicted Emotion: sadness

Input: قلبي ينبض بسرعة وأنا قلق جدا
Predicted Emotion: sadness

Input: أشعر بالخوف من فقدان أحبائي
Predicted Emotion: sadness

Input: لا أستطيع التوقف عن التفكير في الأمور السيئة
Predicted Emotion: sadness

Input: لا أريد التحدث إلى أحد اليوم
Predicted Emotion: sadness

Input: أشعر بالحزن الشديد بعد ما حدث
Predicted Emotion: sadness

Input: لا أستطيع التجاوز عن ما فقدته
Predicted Emotion: sadness

Input: أشعر بالوحدة والحزن العميق
Predicted Emotion: fear

Input: كل شيء مظلم وأشعر باليأس
Predicted Emotion: anger

Input: أنا غاضب من الناس من حولي
Predicted Emotion: anger

Input: أشعر بالغضب الشديد بسبب الظلم
Predicted Emotion: anger

Input: أريد التعبير عن غضبي لكن لا أستطيع
Predicted Emotion: anger

Input: كل شيء يزعجني ويثير غضبي
Predicted Emotion: anger

Input: الغضب يسيطر علي ولا أستطيع التحكم فيه
Predicted Emotion: anger

Input: أنا سعيد جدًا اليوم
P

In [2]:
!pip install gradio


In [3]:
import gradio as gr

def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    pred = torch.argmax(outputs.logits, dim=1).item()
    emotion = id2label[pred]
    return emotion

# Gradio UI
interface = gr.Interface(fn=predict_emotion,
                         inputs="text",
                         outputs="text",
                         title="Arabic Mental Health Emotion Detector",
                         description="Enter Arabic text and detect emotion (fear, sadness, anger, joy, neutral).")

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7285c197d086f35779.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
